In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import cv2
import shutil
import gc
import zipfile
import matplotlib.pyplot as plt
from tqdm import tqdm
from google.colab import drive
from sklearn.model_selection import train_test_split

In [ ]:
drive.mount('/tmp/gdrive')

Mounted at /tmp/gdrive


In [ ]:
local_zip = '/tmp/gdrive/MyDrive/Halitra/bisindo.zip'
zip_ref   = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

In [ ]:
training_dir = os.path.join('/tmp', 'bisindo')

In [ ]:
validation_dataset_portion = 0.1

In [ ]:
!rm -r validation

rm: cannot remove 'validation': No such file or directory


In [ ]:
for class_dir_name in os.listdir(training_dir):
  class_dir_path = os.path.join(training_dir, class_dir_name)
  train_dataset_num = len(os.listdir(class_dir_path))

  val_dataset_num = train_dataset_num * validation_dataset_portion

  val_class_dir_path = os.path.join('/tmp/validation/', class_dir_name)
  os.mkdir(val_class_dir_path)

  counter = 0

  for file_name in os.listdir(class_dir_path):
    filepath = os.path.join(class_dir_path, file_name)
    if counter < val_dataset_num:
      destination = os.path.join(val_class_dir_path, file_name)
      shutil.move(filepath, destination)
      counter += 1

In [ ]:
training = tf.keras.preprocessing.image.ImageDataGenerator (
    rescale = 1./255,
    horizontal_flip = True
)
validation = tf.keras.preprocessing.image.ImageDataGenerator (
    rescale = 1./255,
)

In [ ]:
training_generator = training.flow_from_directory(
    training_dir,
    target_size = (150,150),
    batch_size = 32,
    class_mode = 'categorical',
    shuffle = True
)

validation_generator = validation.flow_from_directory(
    '/tmp/validation',
    target_size = (150,150),
    batch_size = 32,
    class_mode = 'categorical',
    shuffle = True
)

Found 10241 images belonging to 25 classes.
Found 1150 images belonging to 25 classes.


In [ ]:
model = tf.keras.models.Sequential ([
    tf.keras.layers.Conv2D(8, (3, 3), activation = 'elu', input_shape = (150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation = 'elu'),
    tf.keras.layers.Dense(25, activation = 'softmax')
])

In [ ]:
model.compile (
    loss = 'categorical_crossentropy',
    metrics = ['acc'],
    optimizer = 'adam'
)

In [ ]:
model.fit(
    training_generator,
    validation_data = validation_generator,
    epochs = 15
)

Epoch 1/15
321/321 [==============================] - 35s 74ms/step - loss: 3.7066 - acc: 0.0592 - val_loss: 3.1422 - val_acc: 0.0530
Epoch 2/15
321/321 [==============================] - 21s 67ms/step - loss: 3.0560 - acc: 0.0805 - val_loss: 3.0937 - val_acc: 0.0722
Epoch 3/15
321/321 [==============================] - 24s 74ms/step - loss: 2.7498 - acc: 0.1587 - val_loss: 2.5152 - val_acc: 0.2200
Epoch 4/15
321/321 [==============================] - 22s 69ms/step - loss: 2.0263 - acc: 0.3957 - val_loss: 1.8173 - val_acc: 0.4504
Epoch 5/15
321/321 [==============================] - 35s 108ms/step - loss: 1.4540 - acc: 0.5790 - val_loss: 1.3577 - val_acc: 0.6026
Epoch 6/15
321/321 [==============================] - 20s 64ms/step - loss: 1.1427 - acc: 0.6649 - val_loss: 1.2320 - val_acc: 0.6391
Epoch 7/15
321/321 [==============================] - 21s 66ms/step - loss: 0.8636 - acc: 0.7565 - val_loss: 1.0848 - val_acc: 0.6957
Epoch 8/15
321/321 [==============================] - 22s 67m

In [ ]:
model.save('/tmp/gdrive/MyDrive/Halitras/my_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
my_model = tf.keras.models.load_model('/tmp/gdrive/MyDrive/Halitras/my_model.h5')

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(my_model)
tflite_model = converter.convert()

with open('/tmp/gdrive/MyDrive/Halitras/model.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
my_model.get_config()

{'name': 'sequential',
 'layers': [{'module': 'keras.layers',
   'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 150, 150, 3),
    'dtype': 'float32',
    'sparse': False,
    'ragged': False,
    'name': 'conv2d_input'},
   'registered_name': None},
  {'module': 'keras.layers',
   'class_name': 'Conv2D',
   'config': {'name': 'conv2d',
    'trainable': True,
    'dtype': 'float32',
    'batch_input_shape': (None, 150, 150, 3),
    'filters': 8,
    'kernel_size': (3, 3),
    'strides': (1, 1),
    'padding': 'valid',
    'data_format': 'channels_last',
    'dilation_rate': (1, 1),
    'groups': 1,
    'activation': 'selu',
    'use_bias': True,
    'kernel_initializer': {'module': 'keras.initializers',
     'class_name': 'GlorotUniform',
     'config': {'seed': None},
     'registered_name': None},
    'bias_initializer': {'module': 'keras.initializers',
     'class_name': 'Zeros',
     'config': {},
     'registered_name': None},
    'kernel_regularizer': None,


In [ ]:
image = tf.keras.utils.load_img('/tmp/image/A/2021-07-24_19-09-13.jpg', target_size = (150, 150))

In [ ]:
image_array = tf.keras.utils.img_to_array(image)

batch_image_array = np.array([image_array])

result = my_model.predict(batch_image_array)
print(result)

1/1 [==============================] - 0s 92ms/step
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]]


In [ ]:
from google.colab import files
image = files.upload()

Saving 2021-07-24_19-09-13.jpg to 2021-07-24_19-09-13.jpg


In [ ]:
result = my_model.predict(image)
print(result)

In [ ]:
image_array = tf.keras.utils.img_to_array(image)

In [ ]:
image_array.shape

(150, 150, 3)

In [ ]:
batch_image_array = np.array([image_array])

In [ ]:
batch_image_array.shape

(1, 150, 150, 3)

In [ ]:
result = my_model.predict(batch_image_array)

1/1 [==============================] - 0s 19ms/step


In [ ]:
print(result)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  1.]]
